# Quantitative Evaluation der Tufte-Designprinzipien

Dieses Notebook dient der quantitativen Validierung der `dufteplots`-Bibliothek. 
Es vergleicht die Tufte-optimierten Grafiken mit den Standard-Plotnine-Grafiken anhand von berechneten Kenngrößen.

Die dabei ermittelten Kenngrößen sind:

1. **Tintenersparnis (Data-Ink-Ratio)**: Hierbei wird die Reduktion der verwendeten "Tinte" gemessen. Hierfür wird jeder Pixel analysiert und mit der Graustufen-Intensität gewichtet  (0 = Weiß/keine Tinte, 255 = Schwarz/volle Tinte).

2. **Relative Datenfläche (Datendichte)**: Hierbei wird der Anteil der reinen Koordinatensystem-Fläche (Panel Area) an der Gesamtfläche der Grafik (inkl. Ränder und Legenden) gemessen.

3. **Objektklassen-Reduktion (Chartjunk)**: Hierbei wird ein berechneter Score ermittelt, welcher die Anzahl der eingesparten strukturellen Matplotlib-Objektklassen (wie Rahmen, Gitterlinien, Legenden-Boxen oder farbige Hintergründe) im direkten Vergleich zum Standard-Plot misst.

### 1. Setup & Import
Um die Evaluation durchzuführen, wird die lokale `dufteplots`-Bibliothek direkt aus dem Quellcode-Ordner (`src/`) geladen. 


In [1]:
! poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: dufteplots (0.1.2)


In [2]:
from pathlib import Path
from typing import Any, Dict, List

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from plotnine import aes, facet_wrap, geom_boxplot, geom_line, geom_point, ggplot, labs, theme_gray
from IPython.display import display


import dufteplots as dp

# Verhindert, dass Matplotlib versucht, ein Fenster zu öffnen
matplotlib.use("Agg")

# Konfiguration
OUTPUT_STD = Path("standardplots")
OUTPUT_TUFTE = Path("dufteplots")
OUTPUT_STD.mkdir(parents=True, exist_ok=True)
OUTPUT_TUFTE.mkdir(parents=True, exist_ok=True)

# Festgelegte Höhe für alle Plots (in Zoll) und Zufallsseed setzen
FIXED_HEIGHT = 5.0
np.random.seed(42)

### 2. Evaluations-Metriken
Funktionen zur Berechnung der Kennzahlen definieren.


In [3]:
def calculate_ink_intensity(image_path: Path) -> float:
    """
    Berechnet die gewichtete Summe der Pixelintensitäten eines gerenderten Plots.

    Parameters
    ----------
    image_path : Path
        Der Dateipfad zum Plot.

    Returns
    -------
    float
        Die aufsummierte Pixelintensität.
    """
    img = np.array(Image.open(image_path).convert("L"))
    return float(np.sum(255 - img))



def calculate_data_area_ratio(
    plot: Any, width: float, height: float, dpi: int = 300
) -> float:
    """
    Berechnet den Anteil der Datenflächen zur Gesamtfläche des Inputplots.

    Parameters
    ----------
    plot : Any
        Das zu rendernde Plotnine-Objekt.
    width : float
        Die Breite der generierten Grafik in Zoll.
    height : float
        Die Höhe der generierten Grafik in Zoll.
    dpi : int, optional
        Die Auflösung in Dots Per Inch, standardmäßig 300.

    Returns
    -------
    float
        Der prozentuale Anteil der reinen Datenfläche an der Gesamtfläche.
    """
    fig = plot.draw()
    fig.set_size_inches(width, height)
    fig.set_dpi(dpi)
    fig.canvas.draw()

    fig_width, fig_height = fig.get_size_inches()
    total_area = fig_width * fig_height

    data_area = 0.0
    for ax in fig.axes:
        bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        data_area += bbox.width * bbox.height

    plt.close(fig)
    return (data_area / total_area) * 100


# Hilfsfunktion zur Zählung der gerenderten Matplotlib-Objekte
def calculate_element_amount(plot: Any) -> Dict[str, int]:
    """
    Zählt alle gerenderten Matplotlib-Objekte der Grafik.
    Es wird Matplotlibs statt Plotnine gescannt, da Plotnine auf Matplotlib aufbaut 
    und die tatsächlichen gerenderten Elemente dort zu finden sind.

    Parameters
    ----------
    plot : Any
        Das auszuwertende Plotnine-Objekt.

    Returns
    -------
    Dict[str, int]
        Zählwerte der gefundenen Matplotlib-Objektklassen.
    """
    fig = plot.draw()
    profile: Dict[str, int] = {}

    # 1. Globale Figure-Objekte
    fig_elements = {
        "fig_legends": fig.legends,
        "fig_texts": fig.texts,
        "fig_patches": fig.patches,
    }
    for cat, items in fig_elements.items():
        profile[cat] = sum(1 for i in items if i and i.get_visible())

    # 2. Lokale Achsen-Objekte
    for ax in fig.axes:
        ax_elements = {
            "ax_lines": ax.lines,
            "ax_patches": ax.patches,
            "ax_texts": ax.texts,
            "ax_collections": ax.collections,
            "ax_spines": ax.spines.values(),
            "ax_ticks_x": ax.xaxis.get_ticklines(),
            "ax_ticks_y": ax.yaxis.get_ticklines(),
            "ax_grids_x": ax.xaxis.get_gridlines(),
            "ax_grids_y": ax.yaxis.get_gridlines(),
            "ax_legends": [ax.get_legend()] if ax.get_legend() else [],
        }
        for cat, items in ax_elements.items():
            profile[cat] = profile.get(cat, 0) + sum(
                1 for i in items if i and getattr(i, "get_visible", lambda: True)()
            )

        # Hintergrundfarbe ermitteln
        facecolor = ax.patch.get_facecolor()
        if facecolor[:3] != (1.0, 1.0, 1.0) and facecolor[3] > 0:
            profile["ax_colored_backgrounds"] = (
                profile.get("ax_colored_backgrounds", 0) + 1
            )

    plt.close(fig)
    return profile


def calculate_total_reduction_score(std_plot: Any, tufte_plot: Any) -> int:
    """
    Vergleicht die Elementprofile und vergibt Punkte für eingesparte Objektklassen.

    Parameters
    ----------
    std_plot : Any
        Das Standard-Plotnine-Objekt als Baseline.
    tufte_plot : Any
        Das Tufte-optimierte Plotnine-Objekt.

    Returns
    -------
    int
        Der Score, der ausdrückt, in wie vielen Kategorien Komplexität reduziert wurde.
    """
    prof_std = calculate_element_amount(std_plot)
    prof_tufte = calculate_element_amount(tufte_plot)

    score = sum(
        1 for cat, count_std in prof_std.items()
        if (count_std > 0) and prof_tufte.get(cat, 0) < count_std
    )
    return score

### 3. Datengenerierung
Synthetische Datensätze für die Plots generieren. 

In [4]:
# Sparkline und Layered Focus-Daten
currencies = ["EUR/USD", "GBP/USD", "CHF/USD", "AUD/USD"]
df_sparklines = pd.DataFrame([
    {"Currency": c, "Day": t, "Rate": v}
    for c, s in zip(currencies, [1.10, 1.30, 1.05, 0.70])
    for t, v in enumerate(
        s * (1 + np.random.normal(loc=0.0005, scale=0.01, size=60)).cumprod()
    )
 ])

# Scatter-Daten
n_c = 60
gdp = np.exp(np.random.normal(9, 1, n_c))
df_scatter = pd.DataFrame({
    "GDP per Capita ($)": gdp,
    "Life Expectancy (Years)": np.clip(
        45 + 5 * np.log(gdp / 100) + np.random.normal(0, 3, n_c), 50, 85
    ),
})

# Boxplot-Daten
df_range = pd.DataFrame({
    "Experiment": np.repeat(["Expt 1", "Expt 2", "Expt 3"], 40),
    "Velocity_Deviation": np.concatenate([
        np.random.normal(850, 40, 40),
        np.random.normal(790, 25, 40),
        np.random.normal(740, 15, 40),
    ]),
})
df_range["Experiment"] = pd.Categorical(
    df_range["Experiment"], categories=["Expt 1", "Expt 2", "Expt 3"], ordered=True
)

# Slopegraph-Daten
df_slope = pd.DataFrame([
    {"Country": c, "Year": str(y), "Value": v}
    for c, v1, v2 in zip(
        ["CH", "US", "DE", "SE", "JP"],
        [32.5, 31.0, 36.0, 34.0, 29.0],
        [31.0, 33.5, 37.0, 35.0, 30.0],
    )
    for y, v in zip(["1970", "1979"], [v1, v2])
])

# Small Multiples Daten
months = np.arange(1, 13)
rates = [
    8 + 4 * np.cos((months - 1) / 12 * 2 * np.pi),
    5 + 2 * np.exp(-(months - 1)),
    np.linspace(4, 2.5, 12),
    np.full(12, 3.5),
]
df_multi = pd.concat([
    pd.DataFrame({"Month": months, "Rate": r + np.random.normal(0, 0.2, 12), "Sector": s})
    for r, s in zip(rates, ["Const", "Retail", "Tech", "Health"])
])

# Time Series Daten
years = np.arange(1960, 2025, 5)
df_line = pd.DataFrame({
    "Year": years,
    "CO2 (ppm)": 315 + 1.5 * (years - 1960) + 0.01 * (years - 1960) ** 2,
})

### 4. Plotgenerierung

Die Plots konfigurieren, sodass diese mit den Einstellungen erstellt werden können.

In [5]:
plot_configs: List[Dict[str, Any]] = [
    {
        "name": "Layered Focus", "aspect_ratio": 1.9, "filename": "01_layered_focus",
        "std_plot": ggplot(df_sparklines, aes(x="Day", y="Rate", color="Currency")) + geom_line() + labs(title="Euro vs. Currencies") + theme_gray(),
        "tufte_plot": dp.layered_focus(df_sparklines, "Day", "Rate", "Currency", "EUR/USD", title="Euro vs. Currencies"),
    },
    {
        "name": "Dot Dash", "aspect_ratio": 1.0, "filename": "02_dot_dash",
        "std_plot": ggplot(df_scatter, aes(x="GDP per Capita ($)", y="Life Expectancy (Years)")) + geom_point() + labs(title="Wealth & Health") + theme_gray(),
        "tufte_plot": dp.dot_dash_plot(df_scatter, "GDP per Capita ($)", "Life Expectancy (Years)", title="Wealth & Health"),
    },
    {
        "name": "Range Frame", "aspect_ratio": 1.5, "filename": "03_range_frame",
        "std_plot": ggplot(df_range, aes(x="Experiment", y="Velocity_Deviation")) + geom_boxplot() + labs(title="Speed of Light") + theme_gray(),
        "tufte_plot": dp.range_frame(df_range, "Experiment", "Velocity_Deviation", title="Speed of Light"),
    },
    {
        "name": "Slopegraph", "aspect_ratio": 0.9, "filename": "04_slopegraph",
        "std_plot": ggplot(df_slope, aes(x="Year", y="Value", group="Country", color="Country")) + geom_line() + geom_point() + labs(title="Gov Receipts") + theme_gray(),
        "tufte_plot": dp.slopegraph(df_slope, "Country", "Year", "Value", title="Gov Receipts"),
    },
    {
        "name": "Small Multiples", "aspect_ratio": 1.6, "filename": "05_small_multiples",
        "std_plot": ggplot(df_multi, aes(x="Month", y="Rate")) + geom_line() + facet_wrap("~Sector", ncol=2) + labs(title="Unemployment Rate") + theme_gray(),
        "tufte_plot": dp.small_multiples(df_multi, "Month", "Rate", "Sector", ncol=2, title="Unemployment Rate", x_label="Month", y_label="Rate"),
    },
    {
        "name": "Sparklines", "aspect_ratio": 1.2, "filename": "06_sparklines",
        "std_plot": ggplot(df_sparklines, aes(x="Day", y="Rate")) + geom_line() + facet_wrap("~Currency", ncol=1) + labs(title="Currency Trends") + theme_gray(),
        "tufte_plot": dp.sparklines(df_sparklines, "Currency", "Day", "Rate", title="Currency Trends"),
    },
    {
        "name": "Time Series", "aspect_ratio": 1.618, "filename": "07_time_series",
        "std_plot": ggplot(df_line, aes(x="Year", y="CO2 (ppm)")) + geom_line() + geom_point() + labs(title="CO2 Concentration") + theme_gray(),
        "tufte_plot": dp.time_series(df_line, "Year", "CO2 (ppm)", title="CO2 Concentration"),
    },
]

C:\Users\david\OneDrive\Desktop\dufteplots\src\dufteplots\plots\range_frame.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


### 5. Berechnung der Metriken
Berechnung der Metriken für die erstellten Plots.

In [ ]:
results: List[Dict[str, Any]] = []

for config in plot_configs:
    name = config["name"]
    calc_width = FIXED_HEIGHT * config["aspect_ratio"]

    path_std = OUTPUT_STD / f"{config['filename']}_std.png"
    path_tufte = OUTPUT_TUFTE / f"{config['filename']}_tufte.png"

    # Bilder speichern
    config["std_plot"].save(
        path_std, height=FIXED_HEIGHT, width=calc_width, dpi=300, verbose=False
    )
    config["tufte_plot"].save(
        path_tufte, height=FIXED_HEIGHT, width=calc_width, dpi=300, verbose=False
    )

    # Metriken berechnen
    area_std = calculate_data_area_ratio(
        config["std_plot"], calc_width, FIXED_HEIGHT, dpi=300
    )
    area_tufte = calculate_data_area_ratio(
        config["tufte_plot"], calc_width, FIXED_HEIGHT, dpi=300
    )

    ink_std = calculate_ink_intensity(path_std)
    ink_tufte = calculate_ink_intensity(path_tufte)
    ink_red = ((ink_std - ink_tufte) / ink_std) * 100 if ink_std > 0 else 0.0

    footprint_score = calculate_total_reduction_score(
        config["std_plot"], config["tufte_plot"]
)

    results.append({
        "Plot": name,
        "Tinten Std": round(ink_std, 1),
        "Tinten Tufte": round(ink_tufte, 1),
        "Tintenersparnis (%)": round(ink_red, 1),
        "Datenfläche Std (%)": round(area_std, 1),
        "Datenfläche Tufte (%)": round(area_tufte, 1),
        "Flächenzuwachs (%)": round(area_tufte - area_std, 1),
        "Anzahlelemente Std": sum(calculate_element_amount(config["std_plot"]).values()),
        "Anzahlelemente Tufte": sum(calculate_element_amount(config["tufte_plot"]).values()),
        "Objektklassen-Reduktion": footprint_score,
    })

df_results = pd.DataFrame(results)

### 6. Ergebnisse
Für jede der berechnete Kennzahl eine Tabelle mit den Ergebnissen erstellen.

In [ ]:
display("Tabelle 1: Tintenersparnis (Total Ink Reduction)")
display(df_results[["Plot", "Tinten Std", "Tinten Tufte", "Tintenersparnis (%)"]].style.format({
    "Tinten Std": "{:.0f}",
    "Tinten Tufte": "{:.0f}",
    "Tintenersparnis (%)": "{:.1f}%"
}).hide(axis="index"))

display("Tabelle 2: Relative Datenfläche (Datendichte)")
display(df_results[["Plot", "Datenfläche Std (%)", "Datenfläche Tufte (%)", "Flächenzuwachs (%)"]].style.format({
    "Datenfläche Std (%)": "{:.1f}%",
    "Datenfläche Tufte (%)": "{:.1f}%",
    "Flächenzuwachs (%)": "{:+.1f}%"
}).hide(axis="index"))

display("Tabelle 3: Objektklassen-Reduktion (Chartjunk)")
display(df_results[["Plot", "Anzahlelemente Std", "Anzahlelemente Tufte", "Objektklassen-Reduktion"]].style.format({
    "Anzahlelemente Std": "{:.0f}",
    "Anzahlelemente Tufte": "{:.0f}",
    "Objektklassen-Reduktion": "{:.0f}"
}).hide(axis="index"))

'Tabelle 1: Tintenersparnis (Total Ink Reduction)'

Plot,Tinten Std,Tinten Tufte,Tintenersparnis (%)
Layered Focus,62605914,12888264,79.4%
Dot Dash,41521017,11422707,72.5%
Range Frame,54136506,8861255,83.6%
Slopegraph,28319173,10148505,64.2%
Small Multiples,62546416,12781205,79.6%
Sparklines,54442527,11664138,78.6%
Time Series,59914144,10110396,83.1%


'Tabelle 2: Relative Datenfläche (Datendichte)'

Plot,Datenfläche Std (%),Datenfläche Tufte (%),Flächenzuwachs (%)
Layered Focus,67.8%,73.0%,+5.3%
Dot Dash,75.5%,71.0%,-4.5%
Range Frame,76.5%,73.1%,-3.4%
Slopegraph,58.3%,81.5%,+23.2%
Small Multiples,64.4%,57.5%,-7.0%
Sparklines,54.5%,80.5%,+26.0%
Time Series,76.9%,72.2%,-4.7%


'Tabelle 3: Objektklassen-Reduktion (Chartjunk)'

Plot,Anzahlelemente Std,Anzahlelemente Tufte,Objektklassen-Reduktion
Layered Focus,36,31,3
Dot Dash,29,23,3
Range Frame,56,36,6
Slopegraph,38,51,6
Small Multiples,79,39,3
Sparklines,69,63,6
Time Series,28,21,3
